# Querying Multiple Data Formats 
In this notebook, we will cover: 
- How to create and then join BlazingSQL tables from CSV, Parquet, and GPU DataFrame (GDF) sources. 

## Import packages and create Blazing Context
You can think of the BlazingContext much like a Spark Context (i.e. where information such as FileSystems you have registered and Tables you have created will be stored). If you have issues running this cell, restart runtime and try running it again.

In [1]:
# Import RAPIDS AI stack
from blazingsql import BlazingContext
import cudf

bc = BlazingContext()

BlazingContext ready


### Create Table from CSV
Here we create a BlazingSQL table directly from a comma-separated values (CSV) file.

In [3]:
# define column names and types
column_names = ['diagnosis_result', 'radius', 'texture', 'perimeter']
column_types = ['float32', 'float32', 'float32', 'float32']

# identify local directory path (SList)
local_path = !pwd

# create table from CSV file
bc.create_table('data_00', local_path[0] + '/data/cancer_data_00.csv', 
                delimiter=',', dtype=column_types, names=column_names)

### Create Table from Parquet
Here we create a BlazingSQL table directly from an Apache Parquet file.

In [4]:
# create table from Parquet file
bc.create_table('data_01', local_path[0] + '/data/cancer_data_01.parquet')

### Create Table from GPU DataFrame
Here we use cuDF to create a GPU DataFrame (GDF), then use BlazingSQL to create a table from that GDF.

The GDF is the standard memory representation for the RAPIDS AI ecosystem.

In [5]:
# define column names and types
column_names = ['compactness', 'symmetry', 'fractal_dimension']
column_types = ['float32', 'float32', 'float32', 'float32']

# make GDF with cuDF (uses relative path)
gdf_02 = cudf.read_csv('data/cancer_data_02.csv',delimiter=',', dtype=column_types, names=column_names)

# create BlazingSQL table from GDF
bc.create_table('data_02', gdf_02)

# Join Tables Together 

Now we can use BlazingSQL to join all three data formats in a single federated query. 

In [6]:
# define a query
query = '''
        SELECT 
            a.*, b.area, b.smoothness, c.* from main.data_00 as a
            LEFT JOIN data_01 as b
                ON (a.perimeter = b.perimeter)
            LEFT JOIN data_02 as c
                ON (b.compactness = c.compactness)
        '''

# join the tables together
join = bc.sql(query).get()

# extract dataframe
result = join.columns

# display results
result

diagnosis_result  radius  texture  perimeter    area  smoothness  \
0                 1.0    19.0     27.0       72.0   394.0       0.081   
1                 1.0    18.0     25.0       97.0   668.0       0.117   
2                 1.0    23.0     12.0      151.0   954.0       0.143   
3                 0.0     9.0     13.0      133.0  1326.0       0.143   
4                 1.0    21.0     27.0      130.0  1203.0       0.125   
5                 1.0    14.0     16.0       78.0   386.0       0.070   
6                 1.0     9.0     19.0      135.0  1297.0       0.141   
7                 0.0    25.0     25.0       83.0   477.0       0.128   
8                 1.0    19.0     24.0       88.0   520.0       0.127   
9                 1.0    24.0     21.0      103.0   798.0       0.082   
10                0.0    14.0     15.0      132.0  1123.0       0.097   
11                1.0    12.0     13.0       94.0   578.0       0.113   
12                1.0    22.0     19.0       97.0   659.0       0.114   
13                1.0    10.0     16.0       95.0   685.0       0.099   
14                1.0    15.0     14.0      108.0   799.0       0.117   
15                1.0    20.0     14.0      130.0  1203.0       0.125   
16                0.0    17.0     11.0       87.0   566.0       0.098   
17                0.0    16.0     14.0       86.0   520.0       0.108   
18                0.0    17.0     24.0       60.0   274.0       0.102   
19                1.0    20.0     27.0      103.0   798.0       0.082   
20                1.0    19.0     12.0      137.0  1404.0       0.094   
21                1.0     9.0     13.0      110.0   905.0       0.112   
22                1.0    19.0     27.0      116.0   913.0       0.119   
23                1.0    10.0     24.0       97.0   659.0       0.114   
24                1.0    16.0     24.0      122.0  1094.0       0.094   
25                1.0    15.0     15.0      102.0   732.0       0.108   
26                0.0    20.0     21.0       78.0   449.0       0.103   
27                1.0    16.0     26.0      120.0  1040.0       0.095   
28                1.0    15.0     18.0       90.0   578.0       0.119   
29                1.0    25.0     11.0       84.0   476.0       0.119   
..                ...     ...      ...        ...     ...         ...   
286               1.0    10.0     14.0       88.0   559.0       0.102   
287               1.0    24.0     16.0       86.0   563.0       0.082   
288               1.0    20.0     14.0       86.0   563.0       0.082   
289               1.0    14.0     26.0      120.0  1033.0       0.115   
290               1.0    18.0     25.0       97.0   645.0       0.105   
291               1.0    17.0     20.0       96.0   657.0       0.114   
292               1.0    12.0     22.0       96.0   657.0       0.114   
293               0.0    24.0     17.0       59.0   251.0       0.107   
294               1.0    18.0     12.0       94.0   643.0       0.098   
295               0.0    21.0     11.0       83.0   524.0       0.090   
296               1.0    24.0     16.0       86.0   535.0       0.116   
297               1.0    15.0     21.0       87.0   545.0       0.104   
298               1.0    10.0     15.0       85.0   552.0       0.074   
299               1.0    20.0     14.0       86.0   535.0       0.116   
300               0.0    25.0     11.0       87.0   545.0       0.104   
301               1.0    14.0     26.0      120.0  1040.0       0.095   
302               1.0    11.0     21.0      128.0  1104.0       0.091   
303               1.0    10.0     13.0      110.0   870.0       0.096   
304               1.0    11.0     11.0      128.0  1104.0       0.091   
305               1.0    10.0     14.0       88.0   559.0       0.129   
306               1.0    18.0     25.0       97.0   659.0       0.114   
307               0.0    22.0     15.0       83.0   477.0       0.128   
308               0.0    23.0     26.0       54.0   225.0      

# You're Ready to Rock
And... thats it! You are now live with BlazingSQL.

Check out our [docs](https://docs.blazingdb.com) to get fancy or to learn more about how BlazingSQL works with the rest of [RAPIDS AI](https://rapids.ai/).